In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
import requests
import os
import zipfile, io

# Data

### Wrangling

Use the [URL](https://ed-public-download.app.cloud.gov/downloads/CollegeScorecard_Raw_Data.zip) for College Scorecard Raw Data from www.data.gov to download the file: `CollegeScorecare_Raw_Data.zip`. Unzip the data into a folder called `data`.

_Note: Preferably create the folder_`data`_in a parent directory since the extracted data is about 2.5 GB in size. Using Version Control to track such a large folder and pushing it to a repository will soon turn into a headache._ <br><br>
Here is the Python code to do the same:

In [ ]:
folder_name = r'../data'

if not os.path.exists(folder_name):
    os.makedirs(folder_name)


url = r'https://ed-public-download.app.cloud.gov/downloads/CollegeScorecard_Raw_Data.zip'
response = requests.get(url, stream = True)

with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
    for elem in zf.namelist():
        zf.extract(elem, '../data')

For this analysis, we will be using the data for the **Academic Year 2017-18** as reported by all the [Integrated Postsecondary Education Data System (IPEDS)](https://nces.ed.gov/ipeds/) institutions. <br> The data resides in a csv file called: `MERGED2017_18_PP.csv`

In [7]:
df_raw = pd.read_csv('../data/CollegeScorecard_Raw_Data/MERGED2017_18_PP.csv', low_memory = False)

In [9]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7058 entries, 0 to 7057
Columns: 1977 entries, UNITID to OMENRUP_PARTTIME_POOLED_SUPP
dtypes: float64(1894), int64(14), object(69)
memory usage: 106.5+ MB


___________________________________________________________________________________________________________________________

The raw data comprises of 1977 columns. The structure of the data violates the definition of Tidy Data according to Hadely Wickham's Paper on Tidy Data which can be found [here](https://www.jstatsoft.org/article/view/v059i10). <br> Due to this reason, many columns are not variable names; instead, they are possible values. This drastically increases the number of columns in the dataset. In short, the number of variables is far fewer than the number of columns in the dataset.

For this Analysis, we will be selecting a subset of the variables. You can go through the [full documentation](https://collegescorecard.ed.gov/data/documentation/) to figure out which variables strike as most interesting to you.


Here, I have selected the following variables:

- __UNITID__: <br>Data files are provided at the UNITID level, which is the unique identification number assigned to postsecondary institutions as surveyed through IPEDS - Integrated Postsecondary Education Data System.


- __INSTNM__: <br>The institution’s name (INSTNM), as reported in IPEDS.


- __CITY, STABBR__: <br>As reported in IPEDS.


- __NUMBRANCH__: <br>The branch campus column (NUMBRANCH) identifies the number of branch campuses at that institution.


- __HIGHDEG__: <br>Highest award (HIGHDEG) identifies the highest award level conferred at the institution.


- __PREDDEG__: <br>Predominant undergraduate award (PREDDEG) identifies the type of award that the institution primarily confers; for instance, an institution that awards 40 percent bachelor’s degrees, 30 percent associate degrees, and 30 percent certificate programs would be classified as predominantly bachelor’s degree awarding.


- __CONTROL__: <br>This element (CONTROL) is reported directly to IPEDS, and identifies whether the institution’s governance structure is public, private nonprofit, or private for-profit.


- __DISTANCEONLY__: <br>Institutions are identified as distance education-only (DISTANCEONLY) if all their programs are available only via distance education.


- __TUITFTE__: <br>The net tuition revenue per full-time equivalent (FTE) student (TUITFTE) uses tuition revenue minus discounts and allowances, and divides that by the number of FTE undergraduate and graduate students.


- __AVGFACSAL__: <br>The average faculty salary (AVGFACSAL) produces the average faculty salary per month, by dividing the total salary outlays by the number of months worked for all full-time, nonmedical instructional staff.


- __ADM_RATE_ALL__: <br>Colleges report to IPEDS their Fall admissions rate, defined as the number of admitted undergraduates divided by the number of undergraduates who applied. ADM_RATE_ALL represents the admissions rate across all campuses, defined as the total number of admitted undergraduates across all branches divided by the total number of undergraduates who applied across all branches.


- __SATVR25, SATVR75, SATMT25, SATMT75, ACTCM25, ACTCM75__: <br>The files include the 25th and 75th percentiles of SAT reading (SATVR* for _25 and _75), writing (SATWR* for _25 and _75), math (SATMT* for _25 and _75) NOTE: WR is has no data and should be dropped.


- __UGDS__: <br>This element (UGDS) includes the number of degree/certificate-seeking undergraduates enrolled in the fall, as reported in the IPEDS Fall Enrollment component.


- __UG25ABV__: <br>This element identifies the share of students enrolled as of the institutions official fall census date (or October 15 of the IPEDS collection year, whichever is earlier) who are ages 25 and over.


- __PCTFLOAN__: <br>This element (PCTFLOAN), as reported in the IPEDS Student Financial Aid (SFA) component, shows the share of undergraduate students who received federal loans in a given year.


- __CDR3__: <br>Cohort default rates are produced annually32 as an institutional accountability metric; institutions with high default rates may lose access to federal financial aid. The three-year cohort default rate (CDR3) represents a snapshot in time.

Lets select these variables of interest from the `df_raw` and store it in `df`:

In [13]:
cols_of_interest = ['UNITID', 'INSTNM', 'CITY', 'STABBR', 'NUMBRANCH', 'HIGHDEG', 'PREDDEG', 'CONTROL', 'DISTANCEONLY', 
                    'TUITFTE', 'AVGFACSAL', 'ADM_RATE_ALL', 'SATVR25', 'SATVR75', 'SATMT25', 'SATMT75', 'ACTCM25', 
                    'ACTCM75', 'UGDS', 'UG25ABV', 'PCTFLOAN', 'CDR3']

df = df_raw[cols_of_interest]

In [16]:
df.shape

(7058, 22)

### Assessing and Cleaning

In [220]:
df_clean = df.copy()

#### Turn column names to lower case:

In [221]:
df_clean.rename( columns = lambda col: col.strip().lower(), inplace = True )

#### Check for NULL entries:

In [222]:
df_clean.isnull().any()

unitid          False
instnm          False
city            False
stabbr          False
numbranch       False
highdeg         False
preddeg         False
control         False
distanceonly     True
tuitfte          True
avgfacsal        True
adm_rate_all     True
satvr25          True
satvr75          True
satmt25          True
satmt75          True
actcm25          True
actcm75          True
ugds             True
ug25abv          True
pctfloan         True
cdr3             True
dtype: bool

In [223]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7058 entries, 0 to 7057
Data columns (total 22 columns):
unitid          7058 non-null int64
instnm          7058 non-null object
city            7058 non-null object
stabbr          7058 non-null object
numbranch       7058 non-null int64
highdeg         7058 non-null int64
preddeg         7058 non-null int64
control         7058 non-null int64
distanceonly    6614 non-null float64
tuitfte         6593 non-null float64
avgfacsal       4209 non-null float64
adm_rate_all    2230 non-null float64
satvr25         1233 non-null float64
satvr75         1233 non-null float64
satmt25         1233 non-null float64
satmt75         1233 non-null float64
actcm25         1289 non-null float64
actcm75         1289 non-null float64
ugds            6312 non-null float64
ug25abv         6239 non-null float64
pctfloan        6291 non-null float64
cdr3            6055 non-null float64
dtypes: float64(14), int64(5), object(3)
memory usage: 1.2+ MB


There seem to be a lot of NULL values for the SAT and ACT data. We will have to consider this during the analysis.

#### Many Categorical variables have been encoded as Numeric, as evident from the documentation. Let's convert those back:

- **`stabbr`**:

In [224]:
df_clean.stabbr.unique()

array(['AL', 'AK', 'WA', 'AZ', 'NM', 'AR', 'CA', 'MN', 'CO', 'CT', 'NY',
       'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'MI', 'IA', 'KS',
       'MO', 'KY', 'LA', 'ME', 'MD', 'MA', 'MS', 'MT', 'NE', 'NV', 'NH',
       'NJ', 'NC', 'ND', 'OH', 'WV', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD',
       'TN', 'TX', 'UT', 'VT', 'VA', 'WI', 'WY', 'AS', 'GU', 'MP', 'PR',
       'FM', 'PW', 'VI', 'MH'], dtype=object)

In [225]:
df_clean.stabbr.nunique()

59

59? Last time I checked, the US had 50 states. Lets investigate this further. <br>Here is a list of all the states according to Wikipedia:

In [226]:
states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA',
         'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR',
         'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [227]:
len(states)

50

Lets isolate the additional 9 elements in `stabbr`:

In [228]:
set(df_clean.stabbr.unique()) - set(states)

{'AS', 'DC', 'FM', 'GU', 'MH', 'MP', 'PR', 'PW', 'VI'}

A quick Google search shows that these 9 abbrevations are Commonwealth/Territories. <br>We can conclude that all values in the `stabbr` column are valid. 

Next, lets convert this column to type Categorical:

In [229]:
l = list(df_clean.stabbr.unique())

df_clean['stabbr'] = pd.Categorical(df_clean.stabbr, categories = l, ordered = False)

In [230]:
df_clean.stabbr.dtype

CategoricalDtype(categories=['AL', 'AK', 'WA', 'AZ', 'NM', 'AR', 'CA', 'MN', 'CO', 'CT',
                  'NY', 'DE', 'DC', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'MI',
                  'IA', 'KS', 'MO', 'KY', 'LA', 'ME', 'MD', 'MA', 'MS', 'MT',
                  'NE', 'NV', 'NH', 'NJ', 'NC', 'ND', 'OH', 'WV', 'OK', 'OR',
                  'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WI',
                  'WY', 'AS', 'GU', 'MP', 'PR', 'FM', 'PW', 'VI', 'MH'],
                 ordered=False)

- __`highdeg`__ and __`preddeg`__:

In [231]:
df_clean.highdeg.unique()

array([4, 2, 3, 1, 0], dtype=int64)

In [232]:
df_clean.preddeg.unique()

array([3, 2, 1, 4, 0], dtype=int64)

From the documentation, both the columns map the numeric value to categorical value as follows: <br>
    - 4 : Graduate's Degree
    - 3 : Bachelor's Degree
    - 2 : Associate's Degree
    - 1 : Certificate
    - 0 : N/A

In [233]:
df_clean['highdeg'].replace(to_replace = {0: np.nan, 1:'Certificate', 2:"Associate's", 3:"Bachelor's", 
                                                       4:"Graduate's"}, 
                                         inplace = True)

df_clean['preddeg'].replace(to_replace = {0: np.nan, 1:'Certificate', 2:"Associate's", 3:"Bachelor's", 
                                                       4:"Graduate's"}, 
                                         inplace = True)

In [234]:
df_clean.highdeg.value_counts(), df_clean.preddeg.value_counts()

(Certificate    2259
 Graduate's     2047
 Associate's    1513
 Bachelor's      762
 Name: highdeg, dtype: int64, Certificate    2990
 Bachelor's     2095
 Associate's    1217
 Graduate's      308
 Name: preddeg, dtype: int64)

Let's compare the value counts with the original dataframe:<br>
    - 4 : Graduate's Degree
    - 3 : Bachelor's Degree
    - 2 : Associate's Degree
    - 1 : Certificate
    - 0 : N/A

In [235]:
df.HIGHDEG.value_counts(), df.PREDDEG.value_counts()

(1    2259
 4    2047
 2    1513
 3     762
 0     477
 Name: HIGHDEG, dtype: int64, 1    2990
 3    2095
 2    1217
 0     448
 4     308
 Name: PREDDEG, dtype: int64)

Looks fine.

Now lets convert to categorical values according to the mapping:

In [236]:
df_clean['highdeg'] = pd.Categorical(df_clean.highdeg, categories = ['Certificate', "Associate's", "Bachelor's", "Graduate's"],
                                    ordered = True)

In [237]:
df_clean['preddeg'] = pd.Categorical(df_clean.preddeg, categories = ['Certificate', "Associate's", "Bachelor's", "Graduate's"],
                                    ordered = True)

- __`control`__:

In [238]:
df_clean.control.value_counts()

3    2969
1    2063
2    2026
Name: control, dtype: int64

From the documentation, the mapping is as follows:
    - 1 : Public
    - 2 : Private Nonprofit
    - 3 : Private For-Profit

In [239]:
df_clean.control.replace(to_replace = {1: 'Public', 2: 'Private Nonprofit', 3: 'Private For-Profit'}, inplace = True)

In [240]:
df_clean['control'] = pd.Categorical(df_clean.control, categories = ['Public', 'Private Nonprofit', 'Private For-Profit'],
                                    ordered = False)

In [241]:
df_clean.control.value_counts()

Private For-Profit    2969
Public                2063
Private Nonprofit     2026
Name: control, dtype: int64

- **`distanceonly`**:

In [242]:
df_clean.distanceonly.value_counts()

0.0    6561
1.0      53
Name: distanceonly, dtype: int64

From the documentation:
    - 0 : No
    - 1 : Yes

In [243]:
df_clean.distanceonly.replace(to_replace = {0.0: 'No', 1.0: 'Yes'}, inplace = True)

In [244]:
df_clean['distanceonly'] = pd.Categorical(df_clean.distanceonly, categories = ['No', 'Yes'], ordered = False)

In [245]:
df_clean.distanceonly.value_counts()

No     6561
Yes      53
Name: distanceonly, dtype: int64

- **`ugds`**:

UGDS is the number of Undergraduate students that enroll in an academic year. <br>
This variable in the dataset has datatype `float`:

In [246]:
df_clean.ugds.dtype

dtype('float64')

Lets convert this to `int` since the number of students cannot be a float:

In [247]:
pd.Series(df_clean.ugds.unique()).sort_values()

387         0.0
178         1.0
792         2.0
2773        3.0
278         4.0
1225        5.0
935         6.0
2739        7.0
367         8.0
1084        9.0
2661       10.0
220        11.0
1580       12.0
346        13.0
171        14.0
750        15.0
222        16.0
1477       17.0
763        18.0
263        19.0
1516       20.0
1218       21.0
2551       22.0
1245       23.0
108        24.0
499        25.0
440        26.0
120        27.0
160        28.0
925        29.0
         ...   
2322    34180.0
753     35038.0
206     35045.0
564     35127.0
1568    35296.0
210     35620.0
2280    35985.0
2706    36097.0
657     37954.0
1301    38770.0
2333    39965.0
2395    40119.0
2095    40553.0
2325    41182.0
587     41834.0
77      42181.0
1153    42289.0
1931    44853.0
2390    45364.0
91      48666.0
619     51015.0
2331    52568.0
568     56366.0
2277    56976.0
2300    59107.0
895     61700.0
1515    68214.0
2650    72385.0
2793    77269.0
68          NaN
Length: 2816, dtype: flo

In [248]:
df_clean[df_clean.ugds.notna()].ugds

0        4824.0
1       12866.0
2         322.0
3        6917.0
4        4189.0
5       32387.0
6        1404.0
7        2801.0
8        4211.0
9       23391.0
10       1283.0
11       1370.0
12        389.0
13        369.0
14       1365.0
15       4101.0
16       2272.0
17       4349.0
18         83.0
19       3836.0
20       4805.0
21       1217.0
22        537.0
23       1099.0
24         59.0
25        685.0
26        449.0
27       6622.0
28        720.0
29       7052.0
         ...   
6582      314.0
6583       82.0
6585       15.0
6586       55.0
6587       65.0
6588      263.0
6589     8665.0
6590       95.0
6591      151.0
6592      103.0
6593        7.0
6594       64.0
6595      240.0
6596        6.0
6597      283.0
6598       78.0
6599       70.0
6600       48.0
6601       69.0
6602      287.0
6603       67.0
6604       77.0
6605       14.0
6606       72.0
6608       16.0
6609       57.0
6610      501.0
6611        5.0
6612       54.0
6613        8.0
Name: ugds, Length: 6312

In [257]:
df_clean[df_clean.ugds.notna()].ugds

0        4824.0
1       12866.0
2         322.0
3        6917.0
4        4189.0
5       32387.0
6        1404.0
7        2801.0
8        4211.0
9       23391.0
10       1283.0
11       1370.0
12        389.0
13        369.0
14       1365.0
15       4101.0
16       2272.0
17       4349.0
18         83.0
19       3836.0
20       4805.0
21       1217.0
22        537.0
23       1099.0
24         59.0
25        685.0
26        449.0
27       6622.0
28        720.0
29       7052.0
         ...   
6582      314.0
6583       82.0
6585       15.0
6586       55.0
6587       65.0
6588      263.0
6589     8665.0
6590       95.0
6591      151.0
6592      103.0
6593        7.0
6594       64.0
6595      240.0
6596        6.0
6597      283.0
6598       78.0
6599       70.0
6600       48.0
6601       69.0
6602      287.0
6603       67.0
6604       77.0
6605       14.0
6606       72.0
6608       16.0
6609       57.0
6610      501.0
6611        5.0
6612       54.0
6613        8.0
Name: ugds, Length: 6312

In [254]:
df_clean[df_clean.ugds.notna()].ugds = df_clean[df_clean.ugds.notna()].ugds.astype(int, copy = False)

In [255]:
df_clean.ugds.dtype

dtype('float64')

In [256]:
pd.Series(df_clean.ugds.unique()).sort_values()

387         0.0
178         1.0
792         2.0
2773        3.0
278         4.0
1225        5.0
935         6.0
2739        7.0
367         8.0
1084        9.0
2661       10.0
220        11.0
1580       12.0
346        13.0
171        14.0
750        15.0
222        16.0
1477       17.0
763        18.0
263        19.0
1516       20.0
1218       21.0
2551       22.0
1245       23.0
108        24.0
499        25.0
440        26.0
120        27.0
160        28.0
925        29.0
         ...   
2322    34180.0
753     35038.0
206     35045.0
564     35127.0
1568    35296.0
210     35620.0
2280    35985.0
2706    36097.0
657     37954.0
1301    38770.0
2333    39965.0
2395    40119.0
2095    40553.0
2325    41182.0
587     41834.0
77      42181.0
1153    42289.0
1931    44853.0
2390    45364.0
91      48666.0
619     51015.0
2331    52568.0
568     56366.0
2277    56976.0
2300    59107.0
895     61700.0
1515    68214.0
2650    72385.0
2793    77269.0
68          NaN
Length: 2816, dtype: flo

In [190]:
n = np.nan

In [191]:
n

nan

In [197]:

df_clean.query('ugds == NA').ugds

UndefinedVariableError: name 'NA' is not defined